<a href="https://colab.research.google.com/gist/teruyuki-yamasaki/7f0a4159ffe7520981a6f39ad0a042b1/braxenv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[BraxEnvironments.ipynb](https://colab.research.google.com/github/google/brax/blob/main/notebooks/environments.ipynb#scrollTo=_sOmCoOrF0F8)

# Brax Environments

Brax provides [Gym](https://gym.openai.com/)-like environments for training policies.  Just as in Gym, Brax environments have step and reset functions that return agent observations and rewards.  Let's try some out!

In [1]:
#@title Import Brax and some helper modules

import functools
import time

from IPython.display import HTML, Image
import gym

try:
  import brax
except ImportError:
  from IPython.display import clear_output 
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

from brax import envs
from brax import jumpy as jp
from brax.envs import to_torch
from brax.io import html
from brax.io import image
import jax
from jax import numpy as jnp
import torch
v = torch.ones(1, device='cuda')  # init torch cuda before jax

In [Brax Basics](https://colab.research.google.com/github/google/brax/blob/main/notebooks/basics.ipynb) we tried out a few simple physics systems.  In order to train agents to achieve some behavior within these systems, we define an environment: a physics system, a way for the agent to observe the system, and a reward function that defines some objective.  Brax comes with many built-in environments in order to get started quickly.

## OpenAI Gym Environments

**[ant](https://github.com/google/brax/blob/main/brax/envs/ant.py)** from [OpenAI Gym Ant-v2](https://gym.openai.com/envs/Ant-v2/): make a four-legged creature walk forward as fast as possible

**[halfcheetah](https://github.com/google/brax/blob/main/brax/envs/halfcheetah.py)** from [OpenAI Gym HalfCheetah-v2](https://gym.openai.com/envs/HalfCheetah-v2/): make a two-dimensional two-legged creature walk forward as fast as possible

**[hopper](https://github.com/google/brax/blob/main/brax/envs/hopper.py)** from [OpenAI Gym Hopper-v2](https://gym.openai.com/envs/Hopper-v2/): make a two-dimensional one-legged robot hop forward as fast as possible

**[humanoid](https://github.com/google/brax/blob/main/brax/envs/humanoid.py)** from [OpenAI Gym Humanoid-v2](https://gym.openai.com/envs/Humanoid-v2/): make a three-dimensional bipedal robot walk forward as fast as possible, without falling over.

**[reacher](https://github.com/google/brax/blob/main/brax/envs/reacher.py)**: from [OpenAI Gym Reacher-v2](https://gym.openai.com/envs/Reacher-v2/): makes a two-joint reacher arm move its tip to a target.

**[walker2d](https://github.com/google/brax/blob/main/brax/envs/walker2d.py)** from [OpenAI Gym Walker2d-v2](https://gym.openai.com/envs/Walker2d-v2/): make a two-dimensional bipedal robot walk forward as fast as possible

## Custom Environments

**[fetch](https://github.com/google/brax/blob/main/brax/envs/fetch.py)**: make a three-dimensional dog chase after a moving target.

**[grasp](https://github.com/google/brax/blob/main/brax/envs/grasp.py)**: a grabber hand must pick up a ball and carry it to a moving target.

**[ur5e](https://github.com/google/brax/blob/main/brax/envs/ur5e.py)**: a ur5e robot arm that moves its end effector to a series of targets.


In [ ]:
#@title Visualizing pre-included Brax environments { run: "auto" }
#@markdown Select an environment to preview it below:

environment = "ant"  # @param ['ant', 'halfcheetah', 'hopper', 'humanoid', 'reacher', 'walker2d', 'fetch', 'grasp', 'ur5e']
env = envs.create(env_name=environment)
state = env.reset(rng=jp.random_prngkey(seed=0))

HTML(html.render(env.sys, [state.qp]))

In [4]:
#@title Visualizing pre-included Brax environments {run: "auto"}
#@markdown Select an environment to preview it below:

environment = "ant" #@param ['ant', 'halfcheetah', 'hopper', 'humanoid', 'reacher', 'walker2d', 'fetch', 'grasp', 'ur5e']
env = envs.create(env_name=environment) 
state = env.reset(rng=jp.random_prngkey(seed=0))

HTML(html.render(env.sys, [state.qp]))

# Stepping and Jitting Environments

We operate environments via `env.step`:

$$
\text{state}_{t+1} = \text{env.step}(\text{state}_t, \text{action})
$$


The state returned by `env.step` contains:
* the physics state `QP`
* the observation vector
* reward
* whether the environment is done and needs to be reset
* other useful metrics.

By calling `env.step` in sequence we produce a rollout:

In [5]:
%%time
rollout = []
for i in range(100):
  # wiggle sinusoidally
  action = jp.ones((env.action_size,)) * jp.sin(i * jp.pi / 15)
  state = env.step(state, action)
  rollout.append(state)

CPU times: user 43.2 s, sys: 545 ms, total: 43.7 s
Wall time: 45.7 s


In [6]:
%%time 
rollout = [] 
for i in range(100):
    # wiggle sinusoidally 
    action = jp.ones((env.action_size,)) * jp.sin(i * jp.pi / 15) 
    state = env.step(state, action) 
    rollout.append(state) 

CPU times: user 37.9 s, sys: 355 ms, total: 38.3 s
Wall time: 38 s


In order to train policies quickly, we use [JAX](https://github.com/google/jax) to jit compile the `env.step`: 

In [7]:
# jit compile env.step:
state = jax.jit(env.step)(state, jnp.ones((env.action_size,)))

After `env.step` has been compiled, it runs much faster:

In [8]:
%%time
for _ in range(100):
  state = jax.jit(env.step)(state, jnp.ones((env.action_size,)))

CPU times: user 687 ms, sys: 60.7 ms, total: 748 ms
Wall time: 1 s


# Visualizing Rollouts

Brax provides two handy ways to visualize environment rollouts.  The first is an HTML visualizer:

In [9]:
HTML(html.render(env.sys, [s.qp for s in rollout]))